In [2]:
import sys
sys.path.insert(0,'/home/zahradnik/PycharmProjects/master_thesis')

In [3]:
from tsfresh import select_features, extract_features
from tsfresh.utilities.dataframe_functions import impute
from src.modules import conf, fit, raw_data, preprocess, df_columns, evl
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")

/home/zahradnik/PycharmProjects/master_thesis/src/modules/__init__.py
/home/zahradnik/PycharmProjects/master_thesis


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [24, 8]
plt.rcParams['figure.dpi']

100.0

In [6]:
data = fit.load_pcls(
        athlete_name='zahradnik',
        activity_type='running',
        path_to_load=conf["Paths"]["pcl"],
    )

100%|██████████| 386/386 [00:00<00:00, 581.79it/s]


In [7]:
train_df = fit.clean_data(pd.concat(data[0:len(data)-1]))[df_columns]
valid_df = fit.clean_data(pd.concat(data[len(data)-1:]))[df_columns]
speed_train = train_df.enhanced_speed
speed_valid = valid_df.enhanced_speed

In [68]:
len(train_df),len(valid_df), len(train_df.columns), len(valid_df.columns)

(921332, 1401, 12, 12)

In [92]:
math_func = ['mean','sum','prod','max','min']

In [70]:
if len(train_df.columns) > len(valid_df.columns):
    cols = list(valid_df.columns)
else:
    cols = list(train_df.columns)
cols.pop(cols.index('enhanced_speed'))
cols

['heart_rate',
 'distance',
 'enhanced_altitude',
 'cadence',
 'temp',
 'wind_speed',
 'wind_direct',
 'rain',
 'slope_steep',
 'slope_ascent',
 'slope_descent']

In [97]:
train_df=train_df[cols]
valid_df=valid_df[cols]

In [72]:
possibilities = []
for x in cols:
    for y in cols:
        if(x!=y):
            possibilities.append([x,y])
len(possibilities)

110

In [ ]:
from feature_engine.creation import MathFeatures
for x in tqdm_notebook(possibilities):
    for y in math_func:
        features = MathFeatures(variables=x, func=f"{y}")
        train_df = features.fit_transform(train_df)

  0%|          | 0/110 [00:00<?, ?it/s]

In [ ]:
len(train_df.columns)

In [ ]:
for x in tqdm_notebook(possibilities):
    for y in math_func:
        features = MathFeatures(variables=x, func=y)
        train_df = features.fit_transform(train_df)

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(train_df[cols], speed_train)

In [ ]:
y_pred = linreg.predict(valid_df[cols])
y_pred = [x if x > 6 else np.mean(y_pred) for x in y_pred]

In [ ]:
fig = evl.plot(df=valid_df,pred=y_pred,endog='enhanced_speed',true_data=speed_valid)

In [ ]:
print(evl.rmse(actual=speed_valid,predicted=y_pred))